In [77]:
import pandas as pd
import numpy as np
import io
import sagemaker.amazon.common as smac
from pandas import DataFrame
import boto3
from sagemaker import get_execution_role
import sagemaker
from sagemaker import KMeans
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [78]:
role = get_execution_role()
bucket='college-picker-ml-bucket'
dynamo_db1 = 'college-picker-information'
dynamo_db2 = 'college-picker-stat'
print(role)

arn:aws:iam::749428755845:role/service-role/AmazonSageMaker-ExecutionRole-20220402T193429


In [89]:
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
dynamotable = dynamodb_resource.Table(dynamo_db1)

response = dynamotable.scan()
items = response['Items']
base_dict = dict()
for item in items:
#     print(str(id) + " " + item['school_name'] + " " + item['state'] + " " + str(item['num_of_student']) + " " + str(item['sat_scores']))
    dynamodb_data1 = {
        'school_id' : item["school_id"],
        'school_name' : item["school_name"],
        'school_url' : item["school_url"],
        'state': item["state"],
        'num_of_student' : item['num_of_student'],
        'sat_scores' : item['sat_scores'],
        'tuition' : item['tuition'],
        "sum" : 0.0, 
        "num": 0.0,
        "avg": 0.0
    }
    base_dict[(int)(item["school_id"])] = dynamodb_data1
    
    
dynamotable = dynamodb_resource.Table(dynamo_db2)

response = dynamotable.scan()
items = response['Items']
data = []
for item in items:
    key = (int)(item["user_school_id"].split("_")[0])
    base_dict[key]["sum"] += (int)(item["rating"])
    base_dict[key]["num"] += 1
    base_dict[key]["avg"] = base_dict[key]["sum"] / base_dict[key]["num"]

dd = []
for k in sorted(base_dict):
    item = base_dict[k]
    data.append([ item['num_of_student'], item['sat_scores'], item['tuition'], item['avg'] ])
    dd.append([ (float)( item['num_of_student']), (float)( item['sat_scores'] ), (float)( item['tuition'] ), (float)( item['avg'] ) ])

df = pd.DataFrame(data, columns=[ 'num_of_student', 'sat_scores', 'tuition', 'avg' ])
print(df)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


     num_of_student sat_scores tuition  avg
0              5090        959   18634  3.0
1             13549       1245   20400  3.0
2               298          0    6950  3.0
3              7825       1300   23734  3.0
4              3603        938   19396  3.0
...             ...        ...     ...  ...
3995              0          0       0  3.0
3996            229          0       0  3.0
3997            184          0       0  3.0
3998            138          0       0  3.0
3999            555          0       0  3.0

[4000 rows x 4 columns]


In [84]:
data_train = df
data_train = data_train.astype('float32')
num_clusters = 10
output_location = 's3://' + bucket + '/model-artifacts'
kmeans = KMeans(role=role,
               instance_count=1,
               instance_type='ml.c4.xlarge',
               output_path=output_location,
               k=num_clusters)

In [85]:
kmeans.fit(kmeans.record_set(data_train.values), job_name='college-picker-ml-v3')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2022-05-14 10:30:30 Starting - Starting the training job...
2022-05-14 10:30:54 Starting - Preparing the instances for trainingProfilerReport-1652524230: InProgress
.........
2022-05-14 10:32:26 Downloading - Downloading input data
2022-05-14 10:32:26 Training - Downloading the training image......
2022-05-14 10:33:21 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[05/14/2022 10:33:24 INFO 140269227697984] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'init_method': 'random', 'mini_batch_size': '5000', 'epochs': '1', 'extra_center_factor': 'auto', 'local_lloyd_max_iter': '300', 'local_lloyd_tol': '0.0001', 'local_lloyd_init_method': 'kmeans++', 'local_lloyd_num_trials': 'auto', 'half_life_time_size': '0', 'eval_metrics': '["msd"]', 'force_dense': 'true', '_disable_wait_to_read': 'false', '_enable_pro

In [86]:
kmeans_predictor = kmeans.deploy(initial_instance_count=1,
                                 instance_type='ml.m4.xlarge')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


----------!

In [95]:
print(kmeans_predictor)
dynamodb_resource = boto3.resource('dynamodb', region_name='us-east-1')
dynamotable = dynamodb_resource.Table('college-picker-ml')

result = kmeans_predictor.predict(data_train.values[0:len(data_train)])
print(result)

for idx, node in enumerate(result):
    node_cluster = result[idx].label['closest_cluster'].float32_tensor.values[0]
    d1 = result[idx].label['distance_to_cluster'].float32_tensor.values[0]
    d = {
        'school_id' : (str)(idx),
        'closest_cluster' : (int)(node_cluster),
        'distance_to_cluster' : (int)(d1)
    }
    
    try:
        dynamotable.put_item(Item=d)
    except Exception as ex:
        print('---cannot put_item={}'.format(ex))
    

[label {
  key: "closest_cluster"
  value {
    float32_tensor {
      values: 5.0
    }
  }
}
label {
  key: "distance_to_cluster"
  value {
    float32_tensor {
      values: 2502.964599609375
    }
  }
}
, label {
  key: "closest_cluster"
  value {
    float32_tensor {
      values: 7.0
    }
  }
}
label {
  key: "distance_to_cluster"
  value {
    float32_tensor {
      values: 9328.5732421875
    }
  }
}
, label {
  key: "closest_cluster"
  value {
    float32_tensor {
      values: 6.0
    }
  }
}
label {
  key: "distance_to_cluster"
  value {
    float32_tensor {
      values: 2815.9189453125
    }
  }
}
, label {
  key: "closest_cluster"
  value {
    float32_tensor {
      values: 5.0
    }
  }
}
label {
  key: "distance_to_cluster"
  value {
    float32_tensor {
      values: 6889.9951171875
    }
  }
}
, label {
  key: "closest_cluster"
  value {
    float32_tensor {
      values: 5.0
    }
  }
}
label {
  key: "distance_to_cluster"
  value {
    float32_tensor {
      value

In [43]:
clust_movieRatings = ratings.loc[ratings['userId'].isin(users_cluster)]

NameError: name 'ratings' is not defined